<a href="https://colab.research.google.com/github/Zoziologie/ebird2latex/blob/master/script_e2L.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Example of use of eBird2LaTeX python module
In this example, we will see how to use the `e2L` package and produce a checklist pdf. This script is a good place to start learning about the package.

First, let's import the dependancies as well as the `e2L` module [from the github repository](https://github.com/Zoziologie/ebird2latex)

In [1]:
import os
import json
!wget --no-cache --backups=1 "https://raw.githubusercontent.com/Zoziologie/ebird2latex/master/e2L.py"
import e2L

Create project name

In [ ]:
projet_name = "Puerto Rico"
filename = projet_name.replace(' ','_').replace("'",'').replace('.','')

1. Create the Bird List loading data from eBird

In [ ]:
code_loc = 'SE' # See possible country code here : https://confluence.cornell.edu/display/CLOISAPI/eBird-1.1-HotSpotsByRegion + region code for each region ? where ?
lang = ['en_UK','FR'] # a single language, or list of several language en,en_US,de,en_AE,en_AU,en_IN,en_NZ,en_UK,en_ZA,es,es_AR,es_CL,es_CU, es_DO,es_ES,es_MX,es_PA,es_PR,fi,fr,fr_HT,ht_HT,in,is,pt_BR,pt_PT,tr,zh http://help.ebird.org/customer/portal/articles/1596582-common-name-translations-in-ebird
cat = ['species'] # domestic,form,hybrid,intergrade,issf,slash,species,spuh
byear = 1900
eyear = 2015
bmonth = 9
emonth = 10
info, bird_list = e2L.bird_creator(code_loc, lang, cat, byear, eyear, bmonth, emonth)
format='a4paper,margin=.3in'
family = 'true'


barchar url: 
http://ebird.org/barchartData?r=SE&bmo=9&emo=10&byr=1900&eyr=2015&fmt=tsv


ValueError: ignored